#Maria import

In [1]:
import pandas as pd, numpy as np, MySQLdb

In [30]:
df = pd.read_csv('ward_data.csv', encoding = 'ISO-8859-1')
df1 = pd.read_csv('ward-atlas-data.csv', sep = ',', encoding = 'ISO-8859-1')
df2 = pd.read_csv('london-borough-profiles.csv', encoding = 'ISO-8859-1')

df1.drop(0, axis = 0, inplace = True)

###Creating area database

In [89]:
wards_areas = df1[['New Code', 'Borough', 'Names']]
borough_areas = df2.iloc[:33,:3]

#Merging to create ward & borough info for easy aggregation
df_int = wards_areas.merge(borough_areas, how = 'inner', left_on = 'Borough', right_on = 'Area_name')

#Dropping borough info but retaining borough_code as foreign key
wards_areas = df_int.iloc[:, [0,2,3]]
wards_areas.columns = ['Ward_Code', 'Ward', 'Borough_Code']

In [90]:
#Merging to create ward & borough info for easy aggregation
df_int = wards_areas.merge(borough_areas, how = 'inner', left_on = 'Borough', right_on = 'Area_name')

#Dropping borough info but retaining borough_code as foreign key
wards_areas = df_int.iloc[:, [0,2,3]]
wards_areas.columns = ['Ward_Code', 'Ward', 'Borough_Code']

In [121]:
#Building the MariaDB connection
db = MySQLdb.connect(host = 'localhost',
                     user = 'root',
                     passwd = 'TPre3376548triflex',
                     db = 'London_Schema')

cursor = db.cursor()

Creating the borough table

In [83]:
#Creating the boroughs table
cursor.execute("""CREATE TABLE boroughs (
                   BoroughCode VARCHAR (50), 
                   BoroughName VARCHAR(255),
                   LondonRegion VARCHAR(255),
                   CONSTRAINT bcode_pk PRIMARY KEY (BoroughCode))""")

0

In [84]:
#Writing each row of the dataframe into the DB
for row in borough_areas.values:
    cursor.execute('INSERT INTO boroughs(BoroughCode, AreaName, LondonRegion)' \
                   'VALUES("%s", "%s", "%s")', row)

Creating the wards table, note foreign key link into borough table

In [92]:
#Creating the wards table
cursor.execute("""CREATE TABLE wards (
                   WardCode VARCHAR (50),
                   WardName VARCHAR (255),
                   BoroughCode VARCHAR (50),
                   CONSTRAINT wcode_pk PRIMARY KEY (WardCode),
                   CONSTRAINT borough_fk FOREIGN KEY (BoroughCode) REFERENCES boroughs (BoroughCode))""")

0

In [93]:
#Writing wards data to DB
for row in wards_areas.values:
    
    cursor.execute('INSERT INTO wards(WardCode, WardName, BoroughCode)' \
                   'VALUES("%s", "%s", "%s")', row)

**Other variables**

In [101]:
#Population table creation
pop_table1 = df.iloc[:626,[2,3,4,5,6,]]
pop_table1.columns = ['WardCode', 'pop_2015', 'children0-15_2015', 'adults16-64_2015', 'senior65+_2015']

pop_table2 = df1.iloc[:,[1,12,13,41,57,64]]
pop_table2.columns = ['WardCode', 'pop_2023', 'pop_2028', 'pop_2002', 'med_age_2002', 'med_age_2013']

pop_table = pop_table1.merge(pop_table2, on = 'WardCode')

In [102]:
#Creating the population table
cursor.execute("""CREATE TABLE population (
                   WardCode VARCHAR (50), 
                   pop_2015 FLOAT,
                   children015_2015 FLOAT,
                   adults1664_2015 FLOAT,
                   senior65_2015 FLOAT,
                   pop_2023 INT,
                   pop_2028 INT,
                   pop_2002 INT,
                   med_age_2002 INT,
                   med_age_2013 INT,
                   CONSTRAINT ward_fk FOREIGN KEY (WardCode) REFERENCES wards (WardCode))""")

OperationalError: (1050, "Table 'population' already exists")

In [103]:
#Writing pop data
for row in pop_table.values:
    
    cursor.execute('INSERT INTO population(WardCode, pop_2015, children015_2015, adults1664_2015, senior65_2015, pop_2023, pop_2028, pop_2002, med_age_2002, med_age_2013)' \
                   'VALUES("%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s")', row)

Real estate data

In [109]:
#Real estate data
re_table1 = df.iloc[:626, [2,12,30,31,34,35,36,37,38,39,40,62]]
re_table1.columns = ['WardCode', 'Area_sqkm', 'Med_price_14', 'No_props_sold_14', 'pct_detached', 'pct_semid', 'pct_terrace', 'pct_flat', 'pct_owned', 'pct_sr', 'pct_pr', 'pct_area_open']

re_table2 = df1.iloc[:, np.r_[1, 228, 237, 333:347, 861, 864 ]]
re_table2.columns = ['WardCode', 'Med_price_95', 'Med_price_04', 'build_year_all', 'built_01900', 'built_19001918', 'built_19191929', 'built_19301939', 'built_19451954', 'built_19551964', 'built_19651972', 'built_19731982', 'built_19831992', 'built_19931999', 'built_20002009', 'built_20102015', 'built_nkn', 'ptal_2010', 'ptal_2014']

re_table = re_table1.merge(re_table2, on = 'WardCode')

In [122]:
#Creating the RE table
cursor.execute("""CREATE TABLE real_estate (
                   WardCode VARCHAR (50), 
                   area_sqkm FLOAT,
                   med_price_14 FLOAT,
                   no_props_sold_14 FLOAT,
                   pct_detached DOUBLE,
                   pct_semid DOUBLE,
                   pct_terrace DOUBLE,
                   pct_flat DOUBLE,
                   pct_owned DOUBLE,
                   pct_sr DOUBLE,
                   pct_pr DOUBLE,
                   pct_area_open DOUBLE,
                   med_price_95 DOUBLE,
                   med_price_04 DOUBLE,
                   build_year_all DOUBLE,
                   built_01900 DOUBLE,
                   built_19001918 DOUBLE,
                   built_19191929 DOUBLE,
                   built_19301939 DOUBLE,
                   built_19451954 DOUBLE,
                   built_19551964 DOUBLE,
                   built_19651972 DOUBLE,
                   built_19731982 DOUBLE,
                   built_19831992 DOUBLE,
                   built_19931999 DOUBLE,
                   built_20002009 DOUBLE,
                   built_20102015 DOUBLE,
                   built_nkn DOUBLE,
                   ptal_2010 DOUBLE,
                   ptal_2014 DOUBLE,
                   CONSTRAINT ward_fk2 FOREIGN KEY (WardCode) REFERENCES wards (WardCode))""")

0

In [129]:
#Writing real estate data
for row in re_table.values:
    
    cursor.execute('INSERT INTO real_estate(WardCode, area_sqkm, med_price_14, no_props_sold_14, pct_detached, pct_semid, pct_terrace, pct_flat, pct_owned, pct_sr, pct_pr, pct_area_open, med_price_95, med_price_04, build_year_all, built_01900, built_19001918, built_19191929, built_19301939, built_19451954, built_19551964, built_19651972, built_19731982, built_19831992, built_19931999, built_20002009, built_20102015, built_nkn, ptal_2010, ptal_2014)' \
                   'VALUES("%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s")', row)

Demographic data

In [78]:
for i in enumerate(df1.columns.values):
    print(i)

(0, 'Codes')
(1, 'New Code')
(2, 'Borough')
(3, 'Names')
(4, 'Population and Age; Population Estimates and Projections; 2001')
(5, 'Population and Age; Population Estimates and Projections; 2006')
(6, 'Population and Age; Population Estimates and Projections; 2011')
(7, 'Population and Age; Population Estimates and Projections; 2013')
(8, 'Population and Age; Population Estimates and Projections; 2014')
(9, 'Population and Age; Population Estimates and Projections; 2015')
(10, 'Population and Age; Population Estimates and Projections; 2016')
(11, 'Population and Age; Population Estimates and Projections; 2018')
(12, 'Population and Age; Population Estimates and Projections; 2023')
(13, 'Population and Age; Population Estimates and Projections; 2028')
(14, 'Population and Age; Age structure (percentage) - 2013; Aged 0-15')
(15, 'Population and Age; Age structure (percentage) - 2013; Aged 16-64')
(16, 'Population and Age; Age structure (percentage) - 2013; Aged 65+')
(17, 'Population and

(313, 'Dwellings and Council tax; % dwellings in council tax bands F, G or H; 2004')
(314, 'Dwellings and Council tax; % dwellings in council tax bands F, G or H; 2005')
(315, 'Dwellings and Council tax; % dwellings in council tax bands F, G or H; 2006')
(316, 'Dwellings and Council tax; % dwellings in council tax bands F, G or H; 2007')
(317, 'Dwellings and Council tax; % dwellings in council tax bands F, G or H; 2008')
(318, 'Dwellings and Council tax; % dwellings in council tax bands F, G or H; 2009')
(319, 'Dwellings and Council tax; % dwellings in council tax bands F, G or H; 2010')
(320, 'Dwellings and Council tax; % dwellings in council tax bands F, G or H; 2011')
(321, 'Dwellings and Council tax; % dwellings in council tax bands F, G or H; 2015')
(322, 'Property Type; All properties (2015); ')
(323, 'Property Type; Bungalow (2015); ')
(324, 'Property Type; Flat (2015); ')
(325, 'Property Type; Terraced (2015); ')
(326, 'Property Type; Semi-detached (2015); ')
(327, 'Property Ty

(664, 'Crime; Burglary; 2006/07')
(665, 'Crime; Burglary; 2007/08')
(666, 'Crime; Burglary; 2008/09')
(667, 'Crime; Burglary; 2009/10')
(668, 'Crime; Burglary; 2010/11')
(669, 'Crime; Burglary; 2011/12')
(670, 'Crime; Burglary; 2012/13')
(671, 'Crime; Burglary; 2013/14')
(672, 'Crime; Burglary; 2014/15')
(673, 'Crime; Theft and Handling; 2001/02')
(674, 'Crime; Theft and Handling; 2002/03')
(675, 'Crime; Theft and Handling; 2003/04')
(676, 'Crime; Theft and Handling; 2004/05')
(677, 'Crime; Theft and Handling; 2005/06')
(678, 'Crime; Theft and Handling; 2006/07')
(679, 'Crime; Theft and Handling; 2007/08')
(680, 'Crime; Theft and Handling; 2008/09')
(681, 'Crime; Theft and Handling; 2009/10')
(682, 'Crime; Theft and Handling; 2010/11')
(683, 'Crime; Theft and Handling; 2011/12')
(684, 'Crime; Theft and Handling; 2012/13')
(685, 'Crime; Theft and Handling; 2013/14')
(686, 'Crime; Theft and Handling; 2014/15')
(687, 'Crime; Fraud or Forgery; 2001/02')
(688, 'Crime; Fraud or Forgery; 2002

In [80]:
for i in enumerate(df.columns.values):
    print(i)

(0, 'Ward name')
(1, 'Old code')
(2, 'New code')
(3, 'Population - 2015')
(4, 'Children aged 0-15 - 2015')
(5, 'Working-age (16-64) - 2015')
(6, 'Older people aged 65+ - 2015')
(7, '% All Children aged 0-15 - 2015')
(8, '% All Working-age (16-64) - 2015')
(9, '% All Older people aged 65+ - 2015')
(10, 'Mean Age - 2013')
(11, 'Median Age - 2013')
(12, 'Area - Square Kilometres')
(13, 'Population density (persons per sq km) - 2013')
(14, '% BAME - 2011')
(15, '% Not Born in UK - 2011')
(16, '% English is First Language of no one in household - 2011')
(17, 'General Fertility Rate - 2013')
(18, 'Male life expectancy -2009-13')
(19, 'Female life expectancy -2009-13 ')
(20, '% children in reception year who are obese - 2011/12 to 2013/14')
(21, '% children in year 6 who are obese- 2011/12 to 2013/14')
(22, 'Rate of All Ambulance Incidents per 1,000 population - 2014')
(23, 'Rates of ambulance call outs for alcohol related illness - 2014')
(24, 'Number Killed or Seriously Injured on the roads

In [130]:
db.commit()
cursor.close()

In [131]:
db.close()